In [143]:
import yaml
import os
import pandas as pd
# from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
from typing import *
import gensim
# load stopwords
with open('ru_stopwords.yaml', 'r') as f:
    ru_stopwords = yaml.safe_load(f)

In [144]:
def parse_topic(topic):
    num = int(topic[0])
    s = topic[4:-1].split(' + ')
    w = []
    t = []
    for s_i in s:
        w_i, t_i = s_i.split("*")
        w.append(float(w_i))
        t.append(t_i[1:-1])
    return w, t

In [145]:
age_genter_to_num_topics = {(16, 1) : 4,
(17, 1) : 4,
(18, 1) : 4,
(19, 1) : 4,
(20, 1) : 4,
(21, 1) : 4,
(38, 1) : 4,
(39, 1) : 4,
(40, 1) : 4,
(41, 1) : 4,
(42, 1) : 3,
(43, 1) : 3,
(58, 1) : 4,
(59, 1) : 3,
(60, 1) : 3,
(61, 1) : 3,
(62, 1) : 3,
(63, 1) : 4,
(16, 2) : 3,
(17, 2) : 3,
(18, 2) : 4,
(19, 2) : 3,
(20, 2) : 4,
(21, 2) : 3,
(38, 2) : 4,
(39, 2) : 4,
(40, 2) : 3,
(41, 2) : 4,
(42, 2) : 3,
(43, 2) : 3,
(58, 2) : 5,
(59, 2) : 4,
(60, 2) : 3,
(61, 2) : 6,
(62, 2) : 4,
(63, 2) : 4}

In [146]:
dir_path = 'results'

df = []

for file in os.listdir(dir_path):
    if file.endswith('.txt'):
        # print(file[7])
        with open(os.path.join(dir_path, file), 'r', encoding='utf-8') as f:
            df_line = [int(file[16:18]), int(file[19])]
            lines = f.readlines()
            lines = [parse_topic(t) for t in lines]
            # print(lines)
            for topic in lines:
                df_line.append(topic[-1])
            
            if len(df_line) < 7:
                n = 7 - len(df_line)
                df_line += [None] * n
            print(df_line)
            # df.append(pd.Series(df_line), ignore_index=True)
            # df_line = pd.DataFrame(columns=('age', 'gender', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6'), )
            df.append(df_line)

df = pd.DataFrame(df, columns=('age', 'gender', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6'))

[16, 1, ['любить', 'id', 'хороший', 'день', 'любовь', 'друг', 'знать', 'рождение', 'com', 'самый', 'тобой', 'человек', 'http', 'понимать', 'забывать'], None, None, None, None]
[16, 2, ['любить', 'http', 'com', 'друг', 'id', 'хороший', 'день', 'vk', 'ru', 'игра', '_', 'новый', 'писать', 'знать', 'любовь'], None, None, None, None]
[17, 1, ['любить', 'день', 'хороший', 'id', 'любовь', 'рождение', 'знать', 'друг', 'http', 'человек', 'com', 'самый', 'ждать', 'тобой', 'новый'], None, None, None, None]
[17, 2, ['любить', 'http', 'id', 'com', 'день', 'друг', 'ru', 'хороший', '_', 'vk', 'знать', 'новый', 'любовь', 'че', 'рождение'], None, None, None, None]
[18, 1, ['любить', 'день', 'хороший', 'id', 'любовь', 'рождение', 'com', 'http', 'знать', 'друг', 'самый', 'человек', 'любимый', 'ждать', 'каждый'], None, None, None, None]
[18, 2, ['любить', 'http', 'день', 'com', 'id', 'хороший', 'друг', 'любовь', 'ru', '_', 'новый', 'рождение', 'знать', 'vk', 'че'], None, None, None, None]
[19, 1, ['любить

In [147]:
id_to_drop = []
for i in range(len(df)):
    line = df.iloc[i]
    # age_genter_to_num_topics
    if age_genter_to_num_topics[(line.age, line.gender)] != 6 - line.isna().sum():
        id_to_drop.append(i)
df = df.drop(id_to_drop)

In [148]:
df = df.sort_values(['age', 'gender'])

In [149]:
df.to_csv('topics.csv', sep='\t', index=False)

In [150]:
class Encoder:
    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        pass

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        pass

class EncoderElmo(Encoder):
    def __init__(self,
                 options_file: str,
                 weight_file: str,
                 requires_grad: bool = False):
        self.elmo = Elmo(options_file,
                         weight_file,
                         num_output_representations=1,
                         requires_grad=requires_grad)
        self.elmo.eval()
        
    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        out = self.elmo(batch_to_ids([tokens]))["elmo_representations"][0]
        return out[0].mean(dim=0)

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        out = self.elmo(batch_to_ids([tokens]))["elmo_representations"][0]
        return out[0, target_i]
    
class EncoderWord2Vec(Encoder):
    def __init__(self, path: str):
        self.wv = gensim.models.keyedvectors.FastTextKeyedVectors.load(path)

    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        return torch.stack([torch.from_numpy(self.wv.get_vector(token))
                            for token in tokens]).mean(dim=0)

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        emb = torch.stack([torch.from_numpy(self.wv.get_vector(token))
                           for i, token in enumerate(tokens)])
        # return torch.cat((emb[target_i], emb.mean(dim=0)))
        return emb.mean(dim=0)

In [151]:
# path = "ruwikiruscorpora_tokens_elmo_1024_2019"
# elmo = EncoderElmo(
#     options_file=os.path.join(path, "options.json"),
#     weight_file=os.path.join(path, "model.hdf5"),
# )

In [152]:
path = "geowac_tokens_none_fasttextskipgram_300_5_2020"
wv = EncoderWord2Vec(os.path.join(path, "model.model"))

In [153]:
df

,age,gender,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
96,16,1,"[завтра, че, спать, сидеть, ахах, гулять, тя, ...","[любить, хороший, день, рождение, самый, ждать...","[любить, друг, знать, тобой, человек, понимать...","[id, com, http, vk, песня, _, ава, группа, htt...",None,None
65,16,2,"[любить, друг, знать, любовь, думать, человек,...","[хороший, день, новый, рождение, ждать, самый,...","[http, com, id, vk, ru, _, https, группа, лс, ...",None,None,None
98,17,1,"[любить, хороший, знать, друг, любовь, человек...","[день, рождение, новый, пусть, желать, счастие...","[смотреть, че, спать, сидеть, ахах, тя, ночь, ...","[id, http, com, песня, _, vk, группа, фотка, ф...",None,None
67,17,2,"[день, новый, че, рождение, хороший, песня, но...","[http, id, com, ru, _, vk, https, игра, группа...","[любить, друг, знать, любовь, думать, человек,...",None,None,None
100,18,1,"[любить, день, хороший, любовь, рождение, знат...","[id, настроение, лето, ава, приветик, приходит...","[com, http, завтра, _, писать, vk, смотреть, ч...","[песня, ахах, давай, фильм, телефон, принимать...",None,None
101,18,2,"[че, завтра, играть, игра, бля, сука, стена, а...","[http, com, id, ru, _, vk, группа, https, виде...","[любить, друг, хороший, любовь, знать, писать,...","[день, новый, рождение, лс, желать, поздравлят...",None,None
102,19,1,"[id, com, http, _, vk, instagram, фото, ru, ht...","[любить, день, хороший, любовь, рождение, чело...","[ждать, друг, сеичас, думать, завтра, какой, п...","[цена, фильм, размер, видео, ои, принимать, по...",None,None
71,19,2,"[хороший, ждать, думать, новый, че, смотреть, ...","[любить, любовь, друг, знать, человек, каждый,...","[http, день, com, id, ru, рождение, _, vk, htt...",None,None,None
104,20,1,"[писать, фото, цена, группа, лс, размер, телеф...","[любить, хороший, любовь, друг, ждать, любимый...","[день, com, http, рождение, id, _, vk, instagr...","[завтра, помогать, рука, написать, сидеть, пож...",None,None
105,20,2,"[любить, хороший, друг, ждать, знать, каждый, ...","[http, день, com, id, ru, рождение, _, vk, htt...","[любовь, думать, завтра, забывать, такои, пуст...","[новый, писать, группа, песня, игра, новыя, иг...",None,None


# Словарь РуСентиЛекс

In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd


def get_sent(line):
    if line.find('positive/negative') > 0:
        return'positive/negative'
    if line.find('negative') > 0 or line.find('negaitve') > 0:
        return'negative'
    if line.find('positive') > 0 or line.find('postive') > 0 or line.find('postitive') > 0 or line.find('posititve') > 0 or line.find('posiive') > 0:
        return'positive'
    if line.find('neutral') > 0:
        return'neutral'

with open('rusentilex_2017.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()[18:]
    lines = [(get_sent(i), i.split(', ')[0]) for i in lines]
    df = pd.DataFrame(lines).rename(columns={0 : 'sent', 1 : 'word'})
    df['n'] = pd.Series(np.ones(len(df)))
    df = df.groupby(['sent', 'word']).count().reset_index()
    df['mn'] = df.groupby('word')['n'].transform(max)
    df =df[df['mn'] == df['n']]
    df = df.groupby('word').min().reset_index()[['word', 'sent']]
    df = df[df['word'].apply(lambda x: len(x.split(' ')) == 1)]
    sentiment_corpus = defaultdict(list)
    for word, sent in df.values:
        sentiment_corpus[sent].append(word)

sentiment_corpus['negative'].append(sentiment_corpus['positive/negative'])
sentiment_corpus.pop('positive/negative')

for key in sentiment_corpus.keys():
    print(f'{key}: len = {len(sentiment_corpus[key])}, example = {sentiment_corpus[key][:5]}')

negative: len = 7768, example = ['аборт', 'абортивный', 'абракадабра', 'абсурд', 'абсурдность']
positive: len = 2450, example = ['авторитет', 'авторитетность', 'авторитетный', 'адаптационный', 'адаптация']
neutral: len = 498, example = ['байка', 'балласт', 'бархатный', 'безделица', 'безделка']


- Словосочетания удаляем
- Выбираем для слова ту метку, которой больше.
- При равных количествах меток приоритет такой: positive -> neutral -> negative
- positive/negative целиком трансформируется в negative

# Поиск синонимов 

In [48]:
path = "geowac_tokens_none_fasttextskipgram_300_5_2020"
wv = EncoderWord2Vec(os.path.join(path, "model.model"))

In [49]:
words = list(neg_set)[13]
print(f'слово: {words}')
print(f'ближайшие синонимы:')
wv.wv.most_similar(words)

NameError: name 'neg_set' is not defined

In [154]:
a = pd.DataFrame([
    [1, 'positive'],
    [1, 'negative'],
    [1, 'negative'],
    [1, 'positive'],
    [1, 'positive'],
    [1, 'positive'],
    ])

СЛОВАРЬ РУССКИХ СИНОНИМОВ И СХОДНЫХ ПО СМЫСЛУ ВЫРАЖЕНИЙ
Синонимы (от греческого слова «одноимённый») — это слова, совпадающие или близкие по значению. Они выражают одно и то же понятие, но различаются оттенками значения, например: занавесь (общее наименование: полотнище ткани, которым завешивают окна, двери, картины и т.п., отгораживают часть помещения), занавес (большое тяжёлое полотнище, закрывающее дверь, проём и т.п., отгораживающее часть помещения, сцену), занавеска (полотнище из легкой ткани), штора (полотнище ткани для завешивания окна), гардина и портьера (полотнище ткани для завешивания двери). 
Синонимы могут также различаться по выразительности, могут быть книжными и разговорными, современными и устарелыми, просторечными, народно - поэтическими, ласкательными и шутливыми: мальчик (подросток мужского пола), мальчишка (разговорное), мальчуган и мальчонка (разговорное и ласкательное), мальчонок (просторечное, ласкательное), парнишка и паренёк (разговорное и ласкательно-фамильярное), хлопец и хлопчик (просторечное и областное), шпингалет (просторечное), пацан (грубоватопросторечное), шкет (жаргонное). 
Синонимы отделяются друг от друга запятой; точка с запятой разделяет синонимы с разными оттенками значения.

Примеры условных обозначений:                      
- Ср. <Ясный> — сравни со словарной статьей «ясный»           
- Ср. <Ад, Много и Углубление> сравни со статьями «ад», «много» и «углубление»                                   
- Прот. <Райский> — Противоположность статье «Райский»        
- [нем. Abschied отставка, увольнение] — пояснение смысла слова 
- || брать авансом, забирать авансом — устойчивые (типичные) словосочетания, идиомы  
- Пушк. — А. Пушкин, Тург., Тургенев — И.Тургенев и т.д.       
- (Даль, алкать) — словарь В.Даля, статья АЛКАТЬ               


In [155]:
import yaml
with open('ru_stopwords.yaml', 'r') as f:
    ru_stopwords = yaml.safe_load(f)
with open('n-abramov-slovar-russkih-sinonimov-i-shodnyh-po-smyslu-vyrazheniy.txt', 'r', encoding='utf-8') as f:
    lines = []
    while True:
        line = f.readline()
        if not line:
            break
        if line.startswith('      '):
            lines[-1] = lines[-1] + line[5:][:-1]
        else:
            lines.append(line[:-1])

In [156]:
import re
synonyms = []
for i, line in enumerate(lines):
    line = re.split(r'см\.|См\.', line)[0]
    line = re.split(r'\|\|', line)[0]
    line = re.split(r'\[.*\]', line)[0]
    line = re.split(r'\(.*\)', line)[0]
    line = re.split(r'\.', line)[0].split(',')
    new_line = []
    for word in line:
        if len(word.strip().split(' ')) == 1 and word.strip().split(' ')[0] not in ru_stopwords and word.strip().split(' ')[0] != "":
            new_line.append(word.strip().lower())
    if len(new_line) > 1:
        synonyms.append(new_line)
len(synonyms), synonyms

(2346,
 [['абонировать', 'нанимать'],
  ['абсолютный',
   'полный',
   'совершенный',
   'безотносительный',
   'самостоятельный'],
  ['автокефальный', 'автократический'],
  ['автор',
   'сочинитель',
   'составитель',
   'писатель',
   'компилятор',
   'композитор',
   'виновник',
   'творец'],
  ['авторитет', 'компетентность', 'компетенция'],
  ['агент', 'представитель', 'уполномоченный', 'поверенный'],
  ['адвокат', 'стряпчий', 'защитник'],
  ['адский',
   'невыносимый',
   'каторжный',
   'дьявольский',
   'сатанинский',
   'анафемский',
   'инфернальный'],
  ['азарт', 'воодушевление', 'пыл', 'энтузиазм', 'остервенение', 'ярость'],
  ['аккуратный',
   'исправный',
   'исполнительный',
   'щепетильный',
   'точный',
   'корректный',
   'пунктуальный',
   'скрупулезный',
   'педантичный',
   'неукоснительный'],
  ['актер', 'артист'],
  ['алкоголик', 'пьяница', 'пропойца'],
  ['аллюр', 'ход', 'походка', 'бег', 'ступа'],
  ['амулет', 'ладанка', 'талисман'],
  ['анализ', 'разложение', '

In [157]:
syn = pd.DataFrame(synonyms)
syn.to_csv('synonim.csv', index=False, sep='\t')

In [158]:
all_words = set()
for g in synonyms:
    all_words.update(set(g))
len(all_words)

10724

In [159]:
for g in synonyms:
    if "дождь" in g:
        print(g)

['много', 'обильно', 'видимо-невидимо', 'множество', 'бездна', 'воз', 'ворох', 'гибель', 'горы', 'град', 'громада', 'груда', 'дождь', 'изобилие', 'кипа', 'кладезь', 'куча', 'лес', 'миллион', 'мириады', 'море', 'обилие', 'облако', 'орава', 'плеяда', 'поток', 'пропасть', 'прорва', 'пучина', 'страсть', 'туча', 'тьма', 'бесчетное']


In [160]:
# import re
# pattern = r'\[.*\] |\(.*\)|Ср\. |\<.*\>\.|\".*\"'
# for i, line in enumerate(lines):
#     line = re.sub(pattern, '', line).strip()
#     lines[i] = line.split("||")[0].strip() if len(line.split("||")) > 1 else line

# for i, line in enumerate(lines):
#     lines[i] = list(map(lambda x: x.strip(), re.split(r'см\.|См\.|\,|;', line)))

# synonims = []
# for i, line in enumerate(lines):
#     new_line = []
#     for j, word in enumerate(line):
#         if len(word.split(' ')) == 1 and word != "":
#             new_line.append(word.strip().lower())
#     if len(new_line) > 1:
#         synonims.append(new_line)
# synonims

In [161]:
df

,age,gender,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
96,16,1,"[завтра, че, спать, сидеть, ахах, гулять, тя, ...","[любить, хороший, день, рождение, самый, ждать...","[любить, друг, знать, тобой, человек, понимать...","[id, com, http, vk, песня, _, ава, группа, htt...",None,None
65,16,2,"[любить, друг, знать, любовь, думать, человек,...","[хороший, день, новый, рождение, ждать, самый,...","[http, com, id, vk, ru, _, https, группа, лс, ...",None,None,None
98,17,1,"[любить, хороший, знать, друг, любовь, человек...","[день, рождение, новый, пусть, желать, счастие...","[смотреть, че, спать, сидеть, ахах, тя, ночь, ...","[id, http, com, песня, _, vk, группа, фотка, ф...",None,None
67,17,2,"[день, новый, че, рождение, хороший, песня, но...","[http, id, com, ru, _, vk, https, игра, группа...","[любить, друг, знать, любовь, думать, человек,...",None,None,None
100,18,1,"[любить, день, хороший, любовь, рождение, знат...","[id, настроение, лето, ава, приветик, приходит...","[com, http, завтра, _, писать, vk, смотреть, ч...","[песня, ахах, давай, фильм, телефон, принимать...",None,None
101,18,2,"[че, завтра, играть, игра, бля, сука, стена, а...","[http, com, id, ru, _, vk, группа, https, виде...","[любить, друг, хороший, любовь, знать, писать,...","[день, новый, рождение, лс, желать, поздравлят...",None,None
102,19,1,"[id, com, http, _, vk, instagram, фото, ru, ht...","[любить, день, хороший, любовь, рождение, чело...","[ждать, друг, сеичас, думать, завтра, какой, п...","[цена, фильм, размер, видео, ои, принимать, по...",None,None
71,19,2,"[хороший, ждать, думать, новый, че, смотреть, ...","[любить, любовь, друг, знать, человек, каждый,...","[http, день, com, id, ru, рождение, _, vk, htt...",None,None,None
104,20,1,"[писать, фото, цена, группа, лс, размер, телеф...","[любить, хороший, любовь, друг, ждать, любимый...","[день, com, http, рождение, id, _, vk, instagr...","[завтра, помогать, рука, написать, сидеть, пож...",None,None
105,20,2,"[любить, хороший, друг, ждать, знать, каждый, ...","[http, день, com, id, ru, рождение, _, vk, htt...","[любовь, думать, завтра, забывать, такои, пуст...","[новый, писать, группа, песня, игра, новыя, иг...",None,None


In [191]:
from collections import defaultdict, Counter
import random

class Group:
    def __init__(self):
        self.age: int = 0
        self.gender: int = 0 
        self.num_topics: int = 0
        self.topics = defaultdict(list)
        self.synonyms = defaultdict(list)

    def __str__(self):
        s = f"\tGroup\n"
        s += f"age: {self.age}\n"
        s += f"gender: {self.gender}\n"
        s += f"num_topics: {self.num_topics}\n"
        s += f"topics:\n"
        for t in self.topics.keys():
            s += str(self.topics[t]) + "\n"
        s += f"synonyms:\n"
        for t in self.synonyms.keys():
            s += str(self.synonyms[t]) + "\n"
        return s
    
    def __repr__(self) -> str:
        return self.__str__()


groups = []
for i in range(len(df)):
    g = Group()
    line = df.iloc[i]
    g.age = line.age
    g.gender = line.gender
    g.num_topics = 6 - line.isna().sum()
    for col in df.columns[2:]:
        if getattr(line, col) is not None:
            g.topics[col] = getattr(line, col)
    groups.append(g)

In [192]:
print(groups)

[	Group
age: 16
gender: 1
num_topics: 4
topics:
['завтра', 'че', 'спать', 'сидеть', 'ахах', 'гулять', 'тя', 'супер', 'хд', 'ои', 'ходить', 'пойти', 'lt', 'ти', 'gt']
['любить', 'хороший', 'день', 'рождение', 'самый', 'ждать', 'новый', 'любимый', 'такои', 'скучать', 'желать', 'самыя', 'дорогой', 'мама', 'школа']
['любить', 'друг', 'знать', 'тобой', 'человек', 'понимать', 'забывать', 'любовь', 'каждый', 'думать', 'сеичас', 'писать', 'смотреть', 'сердце', 'нужный']
['id', 'com', 'http', 'vk', 'песня', '_', 'ава', 'группа', 'https', 'фото', 'эстафета', 'ru', 'amp', 'приветик', 'принимать']
synonyms:
, 	Group
age: 16
gender: 2
num_topics: 3
topics:
['любить', 'друг', 'знать', 'любовь', 'думать', 'человек', 'че', 'понимать', 'девушка', 'смотреть', 'какой', 'забывать', 'играть', 'такои', 'нужный']
['хороший', 'день', 'новый', 'рождение', 'ждать', 'самый', 'завтра', 'школа', 'желать', 'первый', 'брат', 'удача', 'мир', 'проходить', 'фильм']
['http', 'com', 'id', 'vk', 'ru', '_', 'https', 'групп

In [197]:
def color_word_and_all_synonyms(word, color=None):
    # if color is None:
    #     color = unused_colors.pop()
    word_to_color[word] = color
    for syn_g in synonyms:
        if word in syn_g:
            for syn_word in syn_g:
                if syn_word not in word_to_color.keys():
                    word_to_color[syn_word] = color

def get_synonyms_color(word):
    colors = []
    for syn_g in synonyms:
        if word in syn_g:
            for syn_word in syn_g:
                if syn_word in word_to_color.keys():
                    colors.append(word_to_color[syn_word])
    c = Counter(colors)
    return None if c.most_common() == [] else c.most_common()[0][0]



In [198]:
word_to_color = {}
unused_colors = list(range(124, 231))
color = 0
for g in groups:
    for i in range(g.num_topics):
        t = f'topic_{i + 1}'
        # generate color
        if color != 0:
            if color in list(word_to_color.values()):
                color = unused_colors.pop()
        else:
            color = unused_colors.pop()

        for word in g.topics[t]:
            print(f'watch word {word}')
            if word not in word_to_color.keys():
                if get_synonyms_color(word) is None:
                    color_word_and_all_synonyms(word, color)
                else:
                    color_word_and_all_synonyms(word, get_synonyms_color(word))
            else:
                color_word_and_all_synonyms(word, word_to_color[word])
            print(f'drop syn')
            for syn_g in synonyms:
                if word in syn_g:
                    g.synonyms[t] += syn_g
                    print(f'{word} in synonim group {syn_g}')
                 

watch word завтра
drop syn
watch word че
drop syn
watch word спать
drop syn
watch word сидеть
drop syn
watch word ахах
drop syn
watch word гулять
drop syn
гулять in synonim group ['бездействовать', 'бездельничать', 'покоиться', 'почивать', 'киснуть', 'коснеть', 'кейфовать', 'сибаритствовать', 'гулять', 'праздновать', 'праздничать']
гулять in synonim group ['кутить', 'гулять', 'бражничать', 'жуировать', 'пиршествовать', 'пировать', 'пить', 'пьянствовать']
гулять in synonim group ['пировать', 'пиршествовать', 'бражничать', 'веселиться', 'гулять', 'кутить', 'пьянствовать']
гулять in synonim group ['ходить', 'ступать', 'шагать', 'шествовать', 'грясти', 'брести', 'двигаться', 'подвигаться', 'гулять', 'разгуливать', 'прогуливаться', 'тянуться', 'плестись', 'ползти', 'ковылять', 'семенить', 'слоняться', 'шататься', 'шляться', 'шлепать', 'маршировать']
watch word тя
drop syn
watch word супер
drop syn
watch word хд
drop syn
watch word ои
drop syn
watch word ходить
drop syn
ходить in synonim gro

In [200]:
word_to_color

{'завтра': 230,
 'че': 230,
 'спать': 230,
 'сидеть': 230,
 'ахах': 230,
 'гулять': 230,
 'бездействовать': 230,
 'бездельничать': 230,
 'покоиться': 230,
 'почивать': 230,
 'киснуть': 230,
 'коснеть': 230,
 'кейфовать': 230,
 'сибаритствовать': 230,
 'праздновать': 230,
 'праздничать': 230,
 'кутить': 230,
 'бражничать': 230,
 'жуировать': 230,
 'пиршествовать': 230,
 'пировать': 230,
 'пить': 230,
 'пьянствовать': 230,
 'веселиться': 230,
 'ходить': 230,
 'ступать': 230,
 'шагать': 230,
 'шествовать': 230,
 'грясти': 230,
 'брести': 230,
 'двигаться': 230,
 'подвигаться': 230,
 'разгуливать': 230,
 'прогуливаться': 230,
 'тянуться': 230,
 'плестись': 230,
 'ползти': 230,
 'ковылять': 230,
 'семенить': 230,
 'слоняться': 230,
 'шататься': 230,
 'шляться': 230,
 'шлепать': 230,
 'маршировать': 230,
 'тя': 230,
 'супер': 230,
 'хд': 230,
 'ои': 230,
 'идти': 230,
 'направляться': 230,
 'отправляться': 230,
 'следовать': 230,
 'пойти': 230,
 'lt': 230,
 'ти': 230,
 'gt': 230,
 'любить': 

In [205]:
j = 5
g = groups[j]
for g in groups:
    print(f'\033[38;5;231mGroup age {g.age} gender {g.gender}')
    for i in range(g.num_topics):
        words = g.topics[f'topic_{i}'] #+ g.synonyms[f'topic_{i}']
        s = ""
        for word in words:
            s += "\033[38;5;{}m{} ".format(word_to_color[word], word)
        print(s)

Group age 16 gender 1

завтра че спать сидеть ахах гулять тя супер хд ои ходить пойти lt ти gt 
любить хороший день рождение самый ждать новый любимый такои скучать желать самыя дорогой мама школа 
любить друг знать тобой человек понимать забывать любовь каждый думать сеичас писать смотреть сердце нужный 
Group age 16 gender 2

любить друг знать любовь думать человек че понимать девушка смотреть какой забывать играть такои нужный 
хороший день новый рождение ждать самый завтра школа желать первый брат удача мир проходить фильм 
Group age 17 gender 1

любить хороший знать друг любовь человек самый ждать тобой думать забывать любимый понимать завтра каждый 
день рождение новый пусть желать счастие поздравлять праздник любовь мечта удача новыя сдавать здоровье ои 
смотреть че спать сидеть ахах тя ночь супер играть класс музыка фильм посмотреть вчера дом 
Group age 17 gender 2

день новый че рождение хороший песня новыя желать первый удача купить фильм поздравлять первыя второй 
http id co

In [107]:
for g in groups:
    for i in range(g.num_topics):
        words = g.topics[f'topic_{i}'] + g.synonyms[f'topic_{i}']

In [108]:
text = 'xyu'
for i in range(30, 38):
    print("\033[{}m{}".format(i, text))

xyu
xyu
xyu
xyu
xyu
xyu
xyu
xyu
